In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re
import string
from wordcloud import WordCloud, STOPWORDS
from sklearn.metrics import mean_squared_error
import csv

C:\Users\chris\anaconda4\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\chris\anaconda4\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\chris\anaconda4\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [ ]:
restaurants = pd.read_csv('restaurants.csv')
ratings = pd.read_csv('ratings.csv')
df = restaurants.merge(ratings, how="left", on="RestaurantId")
df.head()

,RestaurantId,Name,UserId,Review,Rating
0,1,aleria,1,A full 5 stars! Phenomenal Michelin ranked res...,5
1,1,aleria,2,We spent great evening here. Cozy restaurant w...,5
2,1,aleria,3,10/10. Aleria was our highlight during our tri...,5
3,1,aleria,4,Great service and fantastic food. We enjoyed e...,5
4,1,aleria,5,Let me put it his way: simply perfect. We hate...,5


In [ ]:
del df['Rating']
del df['RestaurantId']
del df['UserId']
df

,Name,Review
0,aleria,A full 5 stars! Phenomenal Michelin ranked res...
1,aleria,We spent great evening here. Cozy restaurant w...
2,aleria,10/10. Aleria was our highlight during our tri...
3,aleria,Great service and fantastic food. We enjoyed e...
4,aleria,Let me put it his way: simply perfect. We hate...
...,...,...
26203,ydria,"Very good grilled octopus, spinach pie and Sau..."
26204,ydria,"The food is sensarional, the service is great...."
26205,ydria,Risotto was excellent. Wine was an extremely s...
26206,ydria,We decided to eat here for an afternoon snack ...


In [ ]:
df = df.groupby(['Name'])['Review'].apply(','.join).reset_index()
df

,Name,Review
0,aleria,A full 5 stars! Phenomenal Michelin ranked res...
1,alexander_the_great,Excellent food and service. \n\nI have been co...
2,arcadia,Nice restaurant with indoor and outdoor seatin...
3,athenascook,"Πολύ νόστιμο και εξαιρετικής ποιότητας φαγητό,..."
4,atitamos,"Είναι τρομερό, το βρήκα μια μέρα τυχαία που το..."
5,balcony,We were looking for somewhere to eat outside a...
6,dio_dekares,Εξαιρετικό εστιατόριο καθ όλα. Φαγητό με γεύσε...
7,efcharis,"Excellent place to eat. The prices are good, f..."
8,elaea,Service was impeccable by the staff of Elaea M...
9,gods_restaurant,They have great food !! Excelent service!! A ...


In [ ]:
df['index'] = df.index

In [ ]:
df

,Name,Review,index
0,aleria,A full 5 stars! Phenomenal Michelin ranked res...,0
1,alexander_the_great,Excellent food and service. \n\nI have been co...,1
2,arcadia,Nice restaurant with indoor and outdoor seatin...,2
3,athenascook,"Πολύ νόστιμο και εξαιρετικής ποιότητας φαγητό,...",3
4,atitamos,"Είναι τρομερό, το βρήκα μια μέρα τυχαία που το...",4
5,balcony,We were looking for somewhere to eat outside a...,5
6,dio_dekares,Εξαιρετικό εστιατόριο καθ όλα. Φαγητό με γεύσε...,6
7,efcharis,"Excellent place to eat. The prices are good, f...",7
8,elaea,Service was impeccable by the staff of Elaea M...,8
9,gods_restaurant,They have great food !! Excelent service!! A ...,9


In [ ]:
cv = CountVectorizer() 
count_matrix = cv.fit_transform(df["Review"])

In [ ]:
cosine_sim = cosine_similarity(count_matrix)

In [ ]:
# συνάρτηση που επιστέφει το όνομα εφόσον γνωρίζουμε το index
def find_title_from_index(index):
    return df[df.index == index]["Name"].values[0]

# συνάρτηση που επιστέφει το index εφόσον γνωρίζουμε το όνομα
def find_index_from_title(title):
    return df[df.Name == title]["index"].values[0]

In [ ]:
restaurant = "ydria"
restaurant_index = find_index_from_title(restaurant)

In [ ]:
similar_restaurant = list(enumerate(cosine_sim[restaurant_index]))

In [ ]:
sorted_similar_restaurant = sorted(similar_restaurant,key=lambda x:x[1],reverse=True)[1:]

In [ ]:
i=0
recommends = []
for element in sorted_similar_restaurant:
    print(find_title_from_index(element[0]))
    i=i+1
    if i>5:
        break

smile
gods_restaurant
arcadia
tzitzikas_kai_mermigas
alexander_the_great
liondi
